In [7]:
import datetime as dt

import pandas as pd

df = pd.DataFrame({'time_ref': [
    dt.datetime(2023, 1, 1, 0, 30),
    dt.datetime(2023, 1, 1, 0, 30),
    dt.datetime(2023, 1, 1, 1),
    dt.datetime(2023, 1, 1, 2),
    dt.datetime(2023, 1, 1, 2, 15),
    dt.datetime(2023, 1, 1, 2, 16),
    dt.datetime(2023, 1, 1, 4),
],
    'value': [1, 2, 1, 10, 10, 10, 10],
    'type': [0, 0, 0, 0, 0, 0, 0]
})
df = df.set_index(pd.DatetimeIndex(df['time_ref']), drop=True)
by = ['value']
window = '1H'
gb_rolling = df.groupby(by=by).rolling(window=window)

In [8]:
type(gb_rolling)

pandas.core.window.rolling.RollingGroupby

In [19]:
gb_rolling.sum()

/var/folders/41/jhvt_zmd3nz3x5jsqblr221r0000gn/T/ipykernel_15042/3518143510.py:1: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['time_ref'], dtype='object')
  gb_rolling.sum()


type
value time_ref                 
1     2023-01-01 00:30:00   0.0
      2023-01-01 01:00:00   0.0
2     2023-01-01 00:30:00   0.0
10    2023-01-01 02:00:00   0.0
      2023-01-01 02:15:00   0.0
      2023-01-01 02:16:00   0.0
      2023-01-01 04:00:00   0.0

In [9]:
dir(gb_rolling)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_blockwise',
 '_apply_pairwise',
 '_apply_series',
 '_apply_tablewise',
 '_as_index',
 '_attributes',
 '_check_window_bounds',
 '_create_data',
 '_dir_additions',
 '_generate_cython_apply_func',
 '_get_window_indexer',
 '_gotitem',
 '_grouper',
 '_index_array',
 '_insert_on_column',
 '_internal_names',
 '_internal_names_set',
 '_is_protocol',
 '_make_numeric_only',
 '_numba_apply',
 '_obj_with_exclusions',
 '_on',
 '_prep_values',
 '_raise_monotonic_error',
 '_resolve

In [4]:
group_hourly_counts = (
    df.groupby(['value', 'hrs_since_group_start'])
    .agg('count')
    .reset_index()
    .rename(columns={'time_ref': 'type'}))
group_hourly_counts

,value,hrs_since_group_start,type
0,1,0.0,2
1,2,0.0,1
2,10,0.0,3
3,10,2.0,1


In [5]:
res = (
    df.merge(
        group_hourly_counts,
        how='left',
        on=['value', 'hrs_since_group_start'])
    .drop(columns='hrs_since_group_start'))
res

,time_ref,value,type
0,2023-01-01 00:30:00,1,2
1,2023-01-01 00:30:00,2,1
2,2023-01-01 01:00:00,1,2
3,2023-01-01 02:00:00,10,3
4,2023-01-01 02:15:00,10,3
5,2023-01-01 02:16:00,10,3
6,2023-01-01 04:00:00,10,1
